# Simple Transformer

## Training Data

In [1]:
with open('data/tiny_shakespeare.txt', 'r', encoding='utf-8') as input:
    text = input.read()

In [2]:
print(f'Number of characters in text: {len(text)}')

Number of characters in text: 1115394


In [3]:
print(f'First 100 chars:\n\n```\n{text[:100]}\n```')

First 100 chars:

```
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
```


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [7]:
vocab_size

65

In [8]:
''.join(chars)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [9]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

In [11]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: [itos[i] for i in l]

In [13]:
encoded_test = encode("hello world!")
encoded_test

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]

In [14]:
decode(encoded_test)

['h', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd', '!']

In [18]:
data = encode(text)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [20]:
block_size = 8
train_data[:block_size + 1]

[18, 47, 56, 57, 58, 1, 15, 47, 58]